# Families of Solving Algorithms

## A. Local Consistency Algorithms (Filtering / Constraint Propagation)

- Don't explore all possibilities.
- Polynomial time.
- Often incomplete alone (don't guarantee finding all solutions).
- Extremely useful for pruning the search space early.

**Types:**
- Node consistency (NC)
- Arc consistency (AC1, AC3)
- Path consistency (PC)

These algorithms do domain filtering before or during search. They don't assign values to variables permanently or find complete solutions by themselves.

---

## B. Complete Search Algorithms

### 1. Generate & Test (GET) - Naive, brute force
- Generate all assignments D(X₁) × ⋯ × D(Xₙ)
- Test each one against constraints.

### 2. Simple Backtracking (SRA) - Smarter than GET
- Assign variables one by one.
- If violated: backtrack immediately.
- Still detects conflicts late.

### 3. Forward Checking (FC) - Smarter backtracking
- After assigning variable Xᵢ = v:
  - Look ahead to all future variables.
  - Remove incompatible values from their domains.

### 4. Look-Ahead (LA) using Arc Consistency
- After assigning:
  - Run AC3 (or PC-2) on the remaining network.
  - Much deeper filtering than FC.
  - Faster detection of dead-ends.

# GET (Generate and Test) Algorithm

```python
GET(A):
    if all variables are assigned:
        return (A satisfies all constraints)
    
    choose an unassigned variable Xi
    for each value v in D(Xi):
        if GET(A ∪ {(Xi, v)}):
            return True
    
    return False
```

# Simple Backtracking (SRA) Algorithm

```python
SRA(A):
    choose an unassigned variable Xi
    for each value v in D(Xi):
        if {(Xi, v)} is consistent with all previous assignments in A:
            if Xi is last variable: return True
            if SRA(A ∪ {(Xi, v)}): return True
    return False
```

### Improvements vs GET
- Stops immediately when a violation occurs.
- But still detects many conflicts late

# Forward Checking (FC)

```python
FC(A, Dom):
    if all variables assigned: return True

    choose an unassigned variable Xi
    for each value vi in Dom[Xi]:
        
        Dom' = copy(Dom)
        Dom'[Xi] = {vi}

        # forward filtering on future variables
        consistent = True
        for each unassigned Xj ≠ Xi:
            remove from Dom'[Xj] all values incompatible with vi
            if Dom'[Xj] becomes empty:
                consistent = False
                break
        
        if consistent:
            if FC(A ∪ {(Xi, vi)}, Dom'):
                return True
    
    return False
```

### Strength
- Detects inconsistencies earlier than SRA.

### Weakness
- Does not guarantee arc consistency between future variables.

# Arc Consistency: AC1 and AC3

## REVISE

To remove values from the domain of a variable `Xi` that have no supporting value in the domain of a neighbor variable `Xj`.

```python
REVISE(Xi, Xj):
    deleted = False
    for each vi in D[Xi]:
        if no vj in D[Xj] satisfies constraints (Xi, vi) ↔ (Xj, vj):
            remove vi from D[Xi]
            deleted = True
    return deleted
```

## AC1 (Simple but slow)

*Node-consistency = remove from each domain all values that violate unary constraints.*

```python
AC1(X, D, C):
    Node-consistency first
    repeat:
        changed = False
        for every arc (Xi, Xj):
            if REVISE(Xi, Xj): changed = True
    until changed == False
```

Revises all arcs repeatedly until nothing changes.

## AC3

Use a queue containing arcs to process

```python
AC3(X, D, C):
    Node-consistency first
    Q = all arcs (Xi, Xj)

    while Q is not empty:
        pop (Xi, Xj)
        if REVISE(Xi, Xj): # domain of Xi changed
            for every (Xk ≠ Xi and Xk ≠ Xj) where constraint exists between Xk and Xi:
                add (Xk, Xi) to Q
```

### Strengths
- Revises only arcs affected by domain changes.

# Look-Ahead Algorithm using AC3

```python
Look_Ahead(A, (X,D,C)):
    (X,D,C) = AC3((X,D,C))

    if domain empty: return False
    if all domains are singletons: return True

    choose a variable Xi not yet assigned
    for each vi in D[Xi]:
        D' = copy(D)
        D'[Xi] = {vi}

        if Look_Ahead(A ∪ {(Xi,vi)}, (X,D',C)): return True

    return False
```

### Strength
- Very strong pruning: detects inconsistencies that FC cannot.
- Used in modern CSP solvers.